In [ ]:
https://github.com/krocki/dnc/blob/master/rnn-numpy.py
https://blog.csdn.net/tudaodiaozhale/article/details/80464060
    
    
http://www.wildml.com/2015/09/recurrent-neural-networks-tutorial-part-2-implementing-a-language-model-rnn-with-python-numpy-and-theano/
https://github.com/dennybritz/rnn-tutorial-rnnlm/blob/master/RNNLM.ipynb

使用numpy等基本库实现RNN

# 公式
## Notation
输入序列：$x, x^{<i>}$  
输出序列：$y,y^{<i>}$
输入，输出序列中的某个点: $ x^{<t>}, y^{<t>} $  
输入，输出序列的长度: $ T_x^{(i)}, T_y^{(i)} $

## forward
$$
a^{<t>} = tanh( w_{aa}a^{(t-1)} + w_{ax}x^{<t>} + b_a )  \\
\hat y = softmax(w_{ya}^{<t>} a^{<t>} + b_y)
$$

## loss
$$
E(y, \hat y) = - \sum_t y_t log \hat y_t
$$

## backpropagation





In [ ]:
def softmax(x):
    e_x = np.exp( x - np.max(x))
    return e_x / e_x.sum(axis=0)

In [2]:
class RNN(object):
    def __init__(self, hid_dim, inp_dim, out_dim):
        self.w_ax = np.random.uniform(-np.sqrt(1.0/inp_dim), np.sqrt(1.0/inp_dim), (hid_dim, inp_dim))
        self.w_aa = np.random.uniform(-np.sqrt(1.0/hid_dim), np.sqrt(1.0/hid_dim), (hid_dim, hid_dim))
        self.w_ya = np.random.uniform(-np.sqrt(1.0/hid_dim), np.sqrt(1.0/hid_dim), (out_dim, hid_dim))
        self.b_a = np.zeros( (hid_dim,1), dtype=float )
        self.b_y = np.zeros( (out_dim,1), dtype=float )
        self.inp_dim = inp_dim
        self.hid_dim = hid_dim
        self.out_dim = out_dim
        
    def forward(self, x):
        T = len(x)
        a = np.zeros( (T + 1, self.hid_dim), dtype=float )
      
        w_ax = self.w_ax
        w_aa = self.w_aa
        w_ya = self.w_ya
        b_a = self.b_a
        b_y = self.b_y
        y_hat = zeros( (T, self.out_dim), dtype=float )
        for t in range(T):
            a[t] = np.tanh(np.dot(w_aa, a[t-1]) + np.dot(w_ax, x[t]) + b_a)
            z = np.dot(w_ya, a[t]) + b_y
            y_hat[t] = softmax(z)
            
        return [y_hat, a]
            
    def predict(self, x):
        y_hat, _ = self.forward(x)
        return np.argmax(y_hat, axis=1)
    
    def train(x, y, a_prev):
        a = {}
        a[-1] = np.copy(a_prev)
        
            
            